# LFP Signal Processing Tutorial

In this tutorial, you will learn the basics of LFP signal processing.

Have fun!

## Import packages that we need

In [ ]:
import numpy as np
from scipy.signal import butter, lfilter, hilbert, cwt, morlet2, decimate, spectrogram, find_peaks
from scipy import stats
from scipy.io import loadmat
import matplotlib.pyplot as plt


## Load data

LFP data from the CA1 region of the hippocampus has been provided. Let's load the data first. 

In [ ]:
#load data
lfp = loadmat('Session1_nostim.mat')
lfp_nostim = {'sample': lfp['cscDataNew_NS']['sample'][0][0].flatten(),
              'ts':lfp['cscDataNew_NS']['timestamps'][0][0].flatten(),
              'fs': lfp['cscDataNew_NS']['Fs'][0][0].flatten()}

## Pre-processing
The data provided is of sampling frequency 500Hz. We want to downsample to 100Hz to reduce computational resources. It is important to low-pass filter our signal prior to downsampling in order to prevent aliasing effects. Once we downsample to 100Hz, we can analyze frequencies upto the Nyquist frequency of 50Hz. The function decimate in scipy.signal accomplishes this in one go! We use a zero-phase filter to make sure that we don't lose phase information while decimating the signal. 
NOTE: Don't forget to downsample the time array and update the sampling frequency!

In [ ]:
#Signal downsampling
data_ds = decimate(lfp_nostim['sample'], q=5, n=8, ftype='fir', zero_phase=True)
time_ds = lfp_nostim['ts'][::5]
time_ds = time_ds - time_ds[0]
fs_ds = lfp_nostim['fs']/5

## Visualization of oscillations
To visualize oscillations in a frequency band, let us bandpass filter our signal in the frequency band of interest. A Hilbert transform can be used to obtain phase estimates of our signal, as well as obtain amplitude envelopes. Instantaneous phase estimates are better if we use a narrow frequency range. 

Notice the edge effects. In dealing with real data, have a buffer of data on either side of the time range of interest, to avoid edge effects.

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

lowcut = 6.
highcut = 10.

y = butter_bandpass_filter(data_ds, lowcut, highcut, fs_ds, order=6)
analyticSignal = hilbert(y)
amplitudeEvelope = np.abs(analyticSignal)
phase_angles = np.angle(analyticSignal)

plt.figure(figsize=(15,5))
plt.plot(time_ds[:500], y[:500])
plt.plot(time_ds[:500], amplitudeEvelope[:500],'r')
plt.legend(['Filtered LFP','Amplitude Envelope'])
plt.ylabel('Amplitude')
plt.xlabel('Time (sec)')

plt.figure(figsize=(15,5))
plt.plot(time_ds[:500], phase_angles[:500])
plt.legend(['Phase estimates'])
plt.ylabel('Phase')
plt.xlabel('Time (sec)')

## Spectrogram

Time-frequency spectrograms quantify the power of oscillations in different frequencies and across time. Here, we perform a wavelet transform using a morlet wavelet. Let us look at a morlet wavelet.


In [ ]:
M=100 #length of wavelet
w=6. #width parameter recommended to be >=5
f=8 #center frequency in hz
fs=100 #sampling frequency
s=w*fs/(2*f*np.pi) #width of wavelet
wavelet = morlet2(M, s, w) #wavelet is complex
plt.plot(np.real(wavelet)) #plot real part of wavelet
plt.plot(abs(wavelet)) #plot wavelet amplitude envelope
plt.show()

We can now compute the time-frequency spectrogram.

In [ ]:
def compute_spectrogram(data, fs, freqrange, w=6):
    #################################
    #Computes the time-frequency spectrogram using the continuous wavelet transform. Uses morlet wavelet.
    #data: LFP samples
    #fs: Sampling frequency of LFP sample
    #freqrange: Tuple of lower and upper bound of frequencies (lower,upper)
    #w: width of morlet wavelet to use. Default 6
    #Returns phase estimates and power
    #################################
    
    freqs = np.linspace(freqrange[0], freqrange[1], 30)
    s = w*fs / (2*freqs*np.pi) #widths at different frequencies of interest
    power = cwt(data, morlet2, s, w=w)
    cwtm_phase = np.angle(power)
    cwtm_power = np.abs(power)
    
    return cwtm_power, cwtm_phase, freqs

freqrange = (1,20) #specify freqrange as a tuple of (low, high)
cwtm_power, cwtm_phase, freqs = compute_spectrogram(data_ds, fs_ds, freqrange, w=6)

In [ ]:
#Plot the spectrogram
plt.pcolormesh(time_ds[:5000], freqs, cwtm_power[:,:5000], cmap='jet') #only plot first 50 seconds
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')
plt.title('Spectrogram')
plt.clim([0,300]) #set the color limits
plt.colorbar()
plt.show()

## Analysis of prominent frequencies

Which is the most prominent oscillation across time? Average the spectrogram along the time axis and calculate the peak frequency.

In [ ]:
time_avg_power = cwtm_power.mean(1)
peak_ind,_ = find_peaks(time_avg_power)
peak_freq = freqs[peak_ind]
plt.figure()
plt.plot(freqs, time_avg_power)
plt.plot(freqs[peak_ind], time_avg_power[peak_ind],'r*')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')

## Compute coherence spectra

In [ ]:
def compute_coherence(phase_data1, phase_data2, fs):
    assert phase_data1.shape == phase_data2.shape
    phase_difference = phase_data1-phase_data2
    phase_coherence = np.sqrt(np.mean(np.sin(phase_difference), axis=1) **2 + np.mean(np.cos(phase_difference), axis=1) **2)
    mean_phase = stats.circmean(phase_difference, high=np.pi, low=-np.pi, axis=1)
    return phase_coherence, mean_phase

phase_coherence, mean_phase=compute_coherence(cwtm_phase, cwtm_phase+np.pi, fs_ds)
plt.figure()
plt.plot(freqs,phase_coherence)

In [ ]:
f, t, Sxx = spectrogram(data_ds, fs_ds)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.clim([0,3000])
plt.colorbar()
plt.show()